In [2]:
""" Подключение ноутбука к google.colab """
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
""" Подключение tensorboardX """
!pip install tensorboardX

     |████████████████████████████████| 225kB 2.8MB/s 


In [4]:
cd /content/drive/My Drive/humans/data/

/content/drive/My Drive/humans/data


In [0]:
""" Подкдючение необходимых библеотек """

import torch.nn.functional as F
import torch
import PIL
import torch.utils.data as dt
import os
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
to_img = ToPILImage()
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from tensorboardX import SummaryWriter
from tqdm import *
import numpy as np
import csv
from pathlib import Path
from glob import glob
from datetime import datetime
import matplotlib.pyplot as plt

from lib import *

%matplotlib inline

In [0]:
DEVICE_ID = 0
DEVICE = torch.device('cuda:%d' % DEVICE_ID)
torch.cuda.set_device(DEVICE_ID)

In [0]:
class HumanDataset(dt.Dataset):
    """ 
        Human features dataset.  Override torch Dataset class to implements reading from h5 files

    """

    def __init__(self, data_path, mask_path):

        """
        Args:
            data_path (string): Path to the images data files.
            mask_path (string): Path were images masks are placed
        """
        self.files = os.listdir(data_path)
        self.files.sort()
        self.mask_files = os.listdir(mask_path)
        self.mask_files.sort()
        self.data_path = data_path
        self.mask_path = mask_path
        assert (len(self.files) == len(self.mask_files))
        self.preprocess = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.files)



In [0]:
""" Подключение датасетов """ 
ds = HumanDataset(train, train_mask) 
ds_test = HumanDataset(test, test)
ds_valid = HumanDataset(valid, valid_mask) 

In [0]:
net = SegmenterModel1()

In [0]:
""" Класс SoftDiceLoss (Dice coefficient). Используется в обучении сети как метрика """
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth = 1e-5
        num = targets.size(0)
        probs = F.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)
        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score


In [0]:
   """ Обучение сети"""
    
    useCuda =True
    n_epoch = 50
    log = './log/'
    train = './train/'
    train_masks = './train_mask/'
    valid = './valid/'
    valid_masks = './valid_mask'
    tb_writer = SummaryWriter(log_dir='log')
    
    m = SegmenterModel1()
    
    """ Выбор критерия оценки и оптимизатора """
    
    criterion = SoftDiceLoss()
    optimizer = optim.Adam(m.parameters(), lr=0.0001)

    if useCuda == True:
        m = m.cuda()
        criterion= criterion.cuda()
        
    """ Подключение загрузчиков """
    
    dl      = dt.DataLoader(ds, shuffle=True, num_workers=4, batch_size=5)
    dl_valid = dt.DataLoader(ds_valid, shuffle=False, num_workers=4, batch_size=5)
    
    """ Процесс обучения"""

    global_iter = 0
    for epoch in range(0, n_epoch):
        print ("Current epoch: ", epoch)
        epoch_loss = 0
        m.train(True)
        for iter, (i, t) in enumerate(tqdm( dl) ):
            i = Variable(i)
            t = Variable(t)
            if useCuda :
                i = i.cuda()
                t = t.cuda()
            o = m(i)
            loss = criterion(o, t)
            loss.backward()
            optimizer.step()

            global_iter += 1
            epoch_loss += loss.data

        epoch_loss = epoch_loss / float(len(ds))
        print ("Epoch loss", epoch_loss)
        tb_writer.add_scalar('Loss/Train', epoch_loss, epoch)

        print ("Make valid")
        valid_loss = 0
        m.train(False)

        tb_out = np.random.choice(range(0, len(dl_valid)), 3 )
        for iter, (i, t) in enumerate(tqdm(dl_valid)):
            i = Variable(i, volatile = True)
            t = Variable(t, volatile = True)
            if useCuda :
                i = i.cuda()
                t = t.cuda()
            o = m(i)
            loss = criterion(o, t)
            valid_loss += loss.data

            for k, c in enumerate(tb_out):
                if c == iter:
                    tb_writer.add_image('Image/Test_input_%d'%k,  i[0].cpu(), epoch)  # Tensor
                    tb_writer.add_image('Image/Test_target_%d'%k, t[0].cpu(), epoch)  # Tensor
                    tb_writer.add_image('Image/Test_output_%d'%k, o[0].cpu(), epoch)  # Tensor

        valid_loss = valid_loss / float(len(ds_test))
        print ("Valid loss", valid_loss)
        tb_writer.add_scalar('Loss/Valid', valid_loss, epoch)

In [0]:
""" Создание и сохранение масок изображений из папки test"""
it = 0
for it in tqdm(range (100)):
  
  ind = str(it + 1460)
  img = ds_valid[it][0].cuda()
  mask = m(img.unsqueeze(0))
  mask = mask.squeeze(0)
  mask = mask.squeeze(0)
  mask = custom_mask(mask)
  mask1 = PIL.Image.fromarray(mask.astype('uint8'))
  mask1.save("./pred_test/" + ind + ".png")

In [0]:
""" Создание pred_valid_template.csv"""
iter = 0
data_len = ds_test.__len__()
data1 = np.zeros(data_len*2)
data1 = data1.reshape(data_len,2)
data1 = data1.tolist()
for iter in tqdm(range (data_len)):   
  img = ds_test[iter][0].cuda()
  mask = m(img.unsqueeze(0))
  mask = mask.squeeze(0)
  mask = mask.squeeze(0)
  mask = custom_mask(mask)
  data1[iter][0] = ds_test.files[iter].split(".")[0]
  data1[iter][1] = encode_rle(mask)

csv_writer(data1, "./pred_valid_template.csv")


In [0]:
""" Создание html страницы """

paths_to_imgs = sorted(glob("./test/*"))

pred_masks = [np.array(Image.open(path)) for path in sorted(glob("./pred_test/*"))]

_ = get_html(paths_to_imgs, pred_masks, path_to_save="results/final")
